In [1]:
%matplotlib inline
import io, os, sys, types

# this import imports the databaseloader script (which only does TGSS and VLSSR for now)
import Databaseloader as dl

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import tkp.db
from tkp.db.model import Varmetric
from tkp.db.model import Runningcatalog
from tkp.db.model import Newsource
from tkp.db.model import Extractedsource
from tkp.db.model import Image

import logging
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import relationship
#import Tools
#import generic_tools
#import plotting_tools
from scipy.stats import norm
import pylab
pylab.rcParams['legend.loc'] = 'best'
from matplotlib.ticker import NullFormatter
from matplotlib.font_manager import FontProperties
#from astroML import density_estimation

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from scipy.spatial import distance

logging.basicConfig(level=logging.INFO)

In [2]:
engine = 'postgresql'
host = 'vlo.science.uva.nl'
port = 5432
user = 'kmeulen'
password = 'kLu2oepRouv2UfoUPhoU'
database='KmeulenTrap4P23'
websiteURL = 'http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'
latexURL = '\url{http://banana.transientskp.org/r4/vlo_'+database+'/runningcatalog/'
query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING

In [3]:
# this is sqlalchemy script to login to the Banana database
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=database)
db.connect()
session = db.Session()

# Here i get the peak flux and the error on the peak flux from all the sources in the database
fpeak = session.query(Extractedsource.f_peak).all()
fpeake = session.query(Extractedsource.f_peak_err).all()

INFO:tkp.db.database:Database config: postgresql://kmeulen@vlo.science.uva.nl:5432/KmeulenTrap4P23
/home/kmeulen/virtualenv/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
INFO:tkp.db.database:connecting to database...
INFO:tkp.db.database:connected to: postgresql://kmeulen@vlo.science.uva.nl:5432/KmeulenTrap4P23


Query to find the variability parameters of the sources in a specific dataset 

In [4]:
# Here I specifiy which dataset of the database to use
dataset_id = 5

VarParams = session.query(Varmetric,Runningcatalog).select_from(join(Varmetric,Runningcatalog)).filter(Runningcatalog.dataset_id == dataset_id).all()



Make 2d array of all the sources except for the transients.

In [5]:

plotdata = [[VarParams[i].Runningcatalog.id, VarParams[i].Runningcatalog.wm_ra,\
             VarParams[i].Runningcatalog.wm_decl,VarParams[i].Runningcatalog.avg_ra_err,\
             VarParams[i].Runningcatalog.avg_decl_err, VarParams[i].Runningcatalog.datapoints,\
             session.query(Newsource).filter(Newsource.id==VarParams[i].Varmetric.newsource).one().newsource_type,\
             VarParams[i].Varmetric.newsource,VarParams[i].Varmetric.lightcurve_max, session.query(Newsource).filter(Newsource.id==VarParams[i].Varmetric.newsource).one().trigger_xtrsrc.id]\
            for i in range(len(VarParams)) if VarParams[i].Varmetric.newsource != None]


In [6]:
# The de Ruiter distance as shown in the Trap Paper section 4.4 https://arxiv.org/abs/1503.01526 

def CalcDeRuiter(transient,ra2,ra2e,dec2,dec2e):
    ra1 = transient['ra']
    dec1 = transient['dec']
    ra1e = transient['rae']
    dec1e = transient['dece']

    r=(np.sqrt((((-1*ra2 +ra1)**2)*(np.cos((dec1+dec2)/2))**2)/\
                       (ra1e**2+ra2e**2)+((-1*dec2+dec1)**2/(dec1e**2+dec2e**2))))
    return r 

In [7]:
# Comparing the fluxes heavily inspired on the de Ruiter distance as used by Bart's thesis: https://api-alumni.nl/media/uploads/theses/phd/lha-scheers-phd.pdf page 55

def CompareFluxes(transient,pflux2,pflux2e):
    pflux1 = transient['pflux']
    pflux1e = transient['pfluxe']
    
    r = np.sqrt((((-1*pflux2 +pflux1)**2))/\
                       (pflux1e**2+pflux2e**2))
    return r

Function for pruning our candidate list

In [8]:
def SelectorForCandidates(rcheck,ycheck,checklight,deruiter=True):
    """Function for pruning the candidate list using both distance and lightcurve measurements"""
    
    print 'before:'
    print len(Transients.instances)
    oldlen = len(Transients.instances) + 1
    
# #   Repeat until no improvement in the length of the candidate list
    while len(Transients.instances) - oldlen < 0:

        translist = []
#         rlist = []
        oldlen = len(Transients.instances)
        
#         These lists are necessary to compare the candidate transients against each other
        ralist = []
        raelist = []
        declist = []
        decelist = []
        pfluxlist = []
        pfluxelist = []

    # """Check candidate transients against each other"""
    
    
        for i in Transients.instances:
            if not deruiter:
                translist.append(i.radec)
            else:
                ralist.append(i.ra)
                raelist.append(i.rae)
                declist.append(i.dec)
                decelist.append(i.dece)
                pfluxlist.append(i.pflux)
                pfluxelist.append(i.pfluxe)

                
        
#       if we are not using deruiter distance use euclidean (it is slightly outdated and not used alot
#       I would recommend just using the deruiterdistance method which also features the updated fluxcheckmethod)
        if not deruiter:
            index1list = []
            index2list = []
            translist = np.reshape(translist,(len(Transients.instances),2))
            Z = distance.cdist(translist,translist,'euclidean')

        # select on distance measure of 0.06 difference
            for g in range(len(Z)):
                for j in range(len(Z[g])):
                    if Z[g][j] <=0.06 and Z[g][j] !=0.0:
                        index1list.append(g)
                        index2list.append(j)
                    i = 0
                    
# the deletion process for notderuiterdistance                    
            while i < len(index1list):

                if checklight:


                    try:
                        if  Transients.instances[index2list[i]].pflux*0.8 <= Transients.instances[index1list[i]].pflux <= Transients.instances[index2list[i]].pflux*1.2:
                            index1list.remove(index2list[i])
                            Transients.instances[index1list[i]] = []
                    except:
                        pass

                else:
                    index1list.remove(index2list[i])
                    Transients.instances[index1list[i]] = []


                i+=1

            i = 0
            while i < len(Transients.instances):
                if Transients.instances[i]:
                    pass
                else:
                    del Transients.instances[i]
                    i-=1     
                i+=1
                        
                        
#       "here we are using deRuiter to calculate the fluxes

        else:
            h = 0
            while h <(len(Transients.instances)):
#               calculate both the deruiterdistance's and compare the fluxes.
                r = CalcDeRuiter(vars(Transients.instances[h]),np.array(ralist),\
                                 np.array(raelist),np.array(declist),np.array(decelist))
                if checklight:
                    y = CompareFluxes(vars(Transients.instances[h]),np.array(pfluxlist),np.array(pfluxelist))
                jk = 0
                for j in range(len(r)):
                    if jk >= len(Transients.instances):
                        break
                    if checklight:
                        if r[j] <=rcheck and r[j]!= 0.0 and 0 < y[j] <=ycheck:
                            del Transients.instances[h]
                            h-=1
                            jk=0
                            break
                    else:
                        if r[j] <=rcheck and r[j]!= 0.0:
                            del Transients.instances[h]
                            h-=1
                            jk=0
                            break
                    jk+=1
                   
                h+=1

        


    # """Check Candidates against all sources"""
#     a zero candidate that is being tracked if the flux is comparable to the transient 
#     could be real things when it is consistently brighter or fainter than the candidate 1.
#  Talk to Mark about pyc to get flux at specific coordinate ore look at the options. in the pyc-h 

        masterindex = []
        ilist = []
        for i in Transients.instances:
            indexlist = []
            
#             again the outdated euclidean method
            if not deruiter:
                Y = distance.cdist(i.radec,X,'euclidean')

                for g in Y:
                    for j in range(len(g)):
                        if checklight:
                            if g[j] <=0.06 and 0.8*lightcata[j]<=i.pflux <= 1.2*lightcata[j]:
                                indexlist.append(j)
                        else:
                            if g[j] <=0.06:
                                indexlist.append(j)
                
#             the de Ruiter method 
            else:
                r = CalcDeRuiter(vars(i),np.array(racata),np.array(raecata),np.array(deccata),np.array(dececata))
                y = CompareFluxes(vars(i),np.array(lightcata),np.array(lightecata))
                for j in range(len(r)):
                    if checklight:
                        if r[j] <=rcheck and y[j]<=ycheck:
                            indexlist.append(j)
                                
                            
                            
                    else:
                        if r[j] <=rcheck:
                            indexlist.append(j)
            masterindex.append(indexlist)
            
# removing the fake candidates
        i = 0
        j= 0
        while i < len(masterindex):
            if masterindex[i]:
                del Transients.instances[j]
                j-=1
            i+=1
            j+=1

    print 'after:'    
    print len(Transients.instances)



Function for comparing transient candidate against external databases

In [9]:

def CheckOutsideDatabase(data,rcheck,ycheck,checklight,gamma,deruiter = True):
    """ Function for checking against outside database the data entry is for the database rcheck is condition \
where we check deruiter against, ycheck is where we check de flux against and checklight,gamma,deruiter\
are booleans which switch on checking the flux(checklight), checking using error on the flux(gamma) and \
using deruiter distance

"""
    masterindex = []
    savedilist = []
    print 'before:'

    print len(Transients.instances)

    for i in Transients.instances:
        indexlist = []

        if not deruiter:
            Y = distance.cdist(i.radec,data.radec,'euclidean')
            if gamma:
                y = CompareFluxes(vars(i),np.array(data.pflux),np.array(data.pfluxe))
                
                for g in Y:

                    for j in range(len(g)):

                        if g[j] <=0.06 and y[j]>= ycheck:
                            if i.id not in savedilist:
                                FluxVar(vars(i),j)
                                savedilist.append(i.id)
                            indexlist.append(j)

                        elif g[j] <=0.06:
                            indexlist.append(j)
                    
            else:
                for g in Y:

                    for j in range(len(g)):
                        if checklight:

                            if g[j] <=0.06 and (i.pflux <= 0.9*data.pflux[j] or i.pflux>=1.1*data.pflux[j]):
                                if i.id not in savedilist:
                                    FluxVar(vars(i),j)
                                    savedilist.append(i.id)
                                indexlist.append(j)

                            elif g[j] <=0.06:
                                indexlist.append(j)
                        else:
                            if g[j] <=0.06:
                                indexlist.append(j)
        else:

            r = CalcDeRuiter(vars(i),np.array(data.ra),np.array(data.rae),np.array(data.dec),np.array(data.dece))
        
            if gamma:
                y = CompareFluxes(vars(i),np.array(data.pflux),np.array(data.pfluxe))
                
                for j in range(len(r)):

                    if r[j] <=rcheck and y[j]>= ycheck:
                        indexlist.append(j)
                        if i.id not in savedilist:                                
                            FluxVar(vars(i),j)
                            savedilist.append(i.id)
                            
                    elif r[j] <=rcheck:
                        indexlist.append(j)
     
            else:
                for j in range(len(r)):
                    if checklight:
                        if r[j] <=rcheck and (i.pflux <= 0.9*data.pflux[j] or i.pflux>=1.1*data.pflux[j]):
                            indexlist.append(j)
                            if i.id not in savedilist:
                                print data.pflux[j]
                                FluxVar(vars(i))
                                savedilist.append(i.id)
                        elif r[j] <=rcheck:
                            indexlist.append(j)
                            




                    else:
                        if r[j] <=rcheck:
                            indexlist.append(j)

        masterindex.append(indexlist)
        
    i = 0
    j = 0
    while i < len(masterindex):
        if masterindex[i]:
            del Transients.instances[j]
            j-=1
        i+=1
        j+=1

    print 'after:'    
    print len(Transients.instances)
    print len(FluxVar.instances)


Class object for saving our candidate transients, class object for storing objects that have varying flux compared to external database and
Initialize our transients class and other banana sources lists.

(This piece of code has to be rerun everytime you adjust something)

In [10]:

class Transients(object):
    instances = []
    def __init__(self,ra,rae,dec,dece,ids,pflux,pfluxe):
        self.id = ids
        y = np.stack((ra,dec),axis = -1)
        self.ra = ra
        self.rae = rae
        self.dec = dec
        self.dece = dece
        self.pflux = pflux
        self.pfluxe = pfluxe
        self.keys = ['yoloswag']
        self.radec = np.reshape(y,(1,2))
        Transients.instances.append(self)

    def DelFalse(i):
        del Transients.instances[i]

class FluxVar(object):
    """class object for storing objects that have a varying flux compared to the database we are looking in"""
    instances = []
    def __init__(self,params,databaseentry):
        self.id = params['id']
        self.ra = params['ra']
        self.dec = params['dec']
        self.radec = params['radec']
        self.pflux = params['pflux']
        self.entry = databaseentry
        try:
            self.iflux = params['iflux']
        except:
            pass
        FluxVar.instances.append(self)
    

        
racata = []
raecata = []
deccata = []
dececata = []
idcata = []
lightcata = []
lightecata = []
for i in range(len(plotdata)):
    if plotdata[i][6] == 1:
        Transients(plotdata[i][1],plotdata[i][3],plotdata[i][2],plotdata[i][4],plotdata[i][0],fpeak[plotdata[i][-1]-1][0],fpeake[plotdata[i][-1]-1][0])
    else:
        racata.append(plotdata[i][1])
        raecata.append(plotdata[i][3])
        deccata.append(plotdata[i][2])
        dececata.append(plotdata[i][4])
        idcata.append(plotdata[i][0])
        lightcata.append(fpeak[plotdata[i][-1]-1][0])
        lightecata.append(fpeake[plotdata[i][-1]-1][0])
X = np.stack((racata,deccata),axis = -1)

racata = np.array(racata)
raecata = np.array(raecata)
dececata = np.array(dececata)
deccta = np.array(deccata)
idcata = np.array(idcata)
lightcata = np.array(lightcata)
lightecata = np.array(lightecata)


here we call the function for calculating the de ruiter distance between candidates and our found sources and if True is given we also check if the max of the lightcurve of the sources compared fall withing 5 sigma flux of each other (thus making it increasingly likely for   the source to be a sidelobe)

This extra flux comparison is used in order to not potentially write of a faint or bright transient close to a known source.

In [11]:
SelectorForCandidates(rcheck = 3, ycheck = 5,checklight = True, deruiter = True)

before:
602
after:
22


Get all candidates

In [12]:
print "Candidate links"
for i in Transients.instances:
    print websiteURL+str(i.id)

Candidate links
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16306
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10785
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10818
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/7857
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/9033
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/7384
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16130
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16338
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/6955
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/6552
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10758
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10195
http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/6659
http://banana.t

print out in latex format:

In [13]:
for i in Transients.instances:
    print str(i.id) + " " + "&"+ " " + '%0.3f' %i.ra + " " + "&"+ " "+ "%0.3f" %(i.dec) +\
    " " + "&" + " " +latexURL +str(i.id)+"}" +" " + '&' + "  "+"\\" + "\\"

16306 & -175.262 & 50.444 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16306} &  \\
10785 & 182.103 & 49.725 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10785} &  \\
10818 & 182.671 & 49.125 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/10818} &  \\
7857 & 186.054 & 46.337 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/7857} &  \\
9033 & 186.056 & 46.336 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/9033} &  \\
7384 & 186.053 & 46.337 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/7384} &  \\
16130 & -173.524 & 44.939 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16130} &  \\
16338 & -173.522 & 44.938 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runningcatalog/16338} &  \\
6955 & 183.241 & 48.807 & \url{http://banana.transientskp.org/r4/vlo_KmeulenTrap4P23/runnin

# tgss 
is the object which has tggs survey data for the field of P23 loaded into it.
could probably just automate it here but lets leave it for now

In [14]:
tgss = dl.ReadData("tgss")
# print dir(tgss)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
CheckOutsideDatabase(tgss,rcheck = 3,ycheck = 1,checklight = True, gamma = True,deruiter = True)

In [ ]:
print "Candidate links"
for i in Transients.instances:
    print websiteURL+str(i.id)

In [ ]:
print 'Fluxvary links'
for i in FluxVar.instances:
    print websiteURL+str(i.id)
    print 'tgssflux: ' + str(tgss.pflux[i.entry])
#     print tgss.ra[i.entry]
#     print tgss.dec[i.entry]
#     print i.ra
#     print i.dec

## VLSSr

In [ ]:
# Probably need to pip install xlrd for excell support in pandas
vssr = dl.ReadData('vssr')
print vssr.keys

In [ ]:
CheckOutsideDatabase(vssr,rcheck = 3,ycheck = 1,checklight = True, gamma = True,deruiter = False)

In [ ]:
for i in FluxVar.instances:
    print websiteURL+str(i.id)
    print 'vssrflux: ' + str(vssr.pflux[i.entry])
#     print vssr.ra[i.entry]
#     print vssr.dec[i.entry]
#     print i.ra
#     print i.dec

In [102]:
for i in range(len(Transients.instances)):
    if Transients.instances[i].id == 3324:
        print i

In [103]:
r = CompareFluxes(vars(Transients.instances[7]),np.array(lightcata),np.array(lightecata))

IndexError: list index out of range

In [104]:
for j in range(len(lightcata)):
    if 0.594<=lightcata[j]<=0.7143 :
        print j

for i in range(len(r)):
    if r[i] <=1:
        print r[i]
print 'yoloswag'
print r[103]
print r[225]
print r[601]

162
234
381
403
503
533
537
655
742
1201
1328
1337
1412
1510
1532
1626
1661
1672
1853
1874
2029
2035
2040
2127
2268
2359
2412
2420
2612
2641
2721
2803
2828
2843
2852
2854
2956
2996
3009
3025
3058
3227
3260
3291
3300
3336
3353
3460
3478
3501
3554
3860
3918
4091
4363
4378
4381
4391
4448
4461
4670
4758
4786
4805
4843
4851
4862
4926
4934
5092
5109
5143
5205
5224
5251
5263
5281
5285
5303
5317
5336
5353
5459
5520
5556
5631
5637
5678
5714
5791
5816
5909
6215
6297
6360
6450
6523
6532
6609
6635
6643
6725
6805
6837
6856
6858
6953
7091
7120
7247
7260
7365
7435
7537
7685
7771
7786
7896
8032
8126
8199
8439
8483
8509
8661
8674
8857
8885
8916
8930
8974
9076
9099
9105
9253
9388
9389
9395
9421
yoloswag
41.38103683732318
42.05647887630652
41.636468117897714


In [105]:
print idcata[0]

11616


Delete double entries in transient list

Delete candidates that have an euclidean distance measure of 0.04 or less with the rest of the sources